<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/An%C3%A1lisis_de_Datos_CSV_PF_UltimaVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importa pandas
import pandas as pd

# Carga los archivos CSV (ajusta el nombre de archivo si es necesario)
df_mi = pd.read_csv('/content/PuntosFijosMI_20250611.csv')
df_md = pd.read_csv('/content/PuntosFijosMD_20250611.csv')

# Carga los archivos XLSX (ajusta el nombre de archivo si es necesario)
#df_mi = pd.read_excel('/content/PuntosFijosMI_20250611.xlsx')
#df_md = pd.read_excel('/content/PuntosFijosMD_20250611.xlsx')

# Muestra las primeras filas de cada DataFrame
print("Primeras filas de PuntosFijosMI:")
display(df_mi.head())

print("Primeras filas de PuntosFijosMD:")
display(df_md.head())

# Muestra información básica de ambos DataFrames
print("Información de PuntosFijosMI:")
df_mi.info()

print("Información de PuntosFijosMD:")
df_md.info()

In [1]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
import io

# --- WIDGET DE SUBIDA DE ARCHIVOS ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightgreen'}
)

output = widgets.Output()

# Variables globales para guardar los DataFrames
df_mi = pd.DataFrame()
df_md = pd.DataFrame()

def cargar_archivos(change):
    global df_mi, df_md
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()

                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
                    continue

                df.columns = df.columns.str.strip()

                # Detección de margen a partir del nombre
                if 'mi' in nombre_archivo.lower():
                    df_mi = pd.concat([df_mi, df], ignore_index=True)
                    print(f"✅ Cargado en df_mi: {nombre_archivo}")
                elif 'md' in nombre_archivo.lower():
                    df_md = pd.concat([df_md, df], ignore_index=True)
                    print(f"✅ Cargado en df_md: {nombre_archivo}")
                else:
                    print(f"⚠️ No se detectó margen en el nombre: {nombre_archivo}")

            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        # Vista previa
        if not df_mi.empty:
            print("\n📄 Vista previa df_mi:")
            display(df_mi.head())
        if not df_md.empty:
            print("\n📄 Vista previa df_md:")
            display(df_md.head())

upload_widget.observe(cargar_archivos, names='value')

# --- MOSTRAR WIDGET E INTERFAZ ---
display(upload_widget)
display(output)


FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

In [2]:
%pip install ipywidgets plotly
from google.colab import output as colab_output
colab_output.enable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.7 MB/s eta 0:00:00


In [3]:
%pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.4 MB/s eta 0:00:00


In [ ]:
print("=== DEBUG MANUAL ===")
print("df_mi shape:", df_mi.shape)
print("df_mi columns:", df_mi.columns.tolist())
print("df_mi INSTRUMENTO unique:", df_mi['INSTRUMENTO'].unique()[:10])  # Primeros 10
print("\ndf_md shape:", df_md.shape)
print("df_md columns:", df_md.columns.tolist())
print("df_md INSTRUMENTO unique:", df_md['INSTRUMENTO'].unique()[:10])  # Primeros 10

In [4]:
import pandas as pd
try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# --- Verificar existencia de DataFrames ---
try:
    df_mi
    df_md
except NameError:
    print("❌ Los DataFrames 'df_mi' y 'df_md' no están definidos. Por favor, ejecuta primero el script de carga de datos.")
    df_mi = pd.DataFrame()
    df_md = pd.DataFrame()

# --- Definiciones comunes ---
estilos_grafico = [
    "Curvas suaves (spline)", "Líneas rectas", "Puntos", "Líneas + Puntos",
    "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
]

tamanios_imagen = {
    "Pequeño (600x400)": (600, 400),
    "Mediano (900x500)": (900, 500),
    "Grande (1200x700)": (1200, 700),
    "Extra grande (1600x1000)": (1600, 1000)
}

grosores = {
    "Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4,
    "Grueso (7px)": 7, "Extra grueso (10px)": 10
}

paletas = {
    "Plotly": pc.qualitative.Plotly, "D3": pc.qualitative.D3,
    "Viridis": pc.sequential.Viridis, "Cividis": pc.sequential.Cividis,
    "Inferno": pc.sequential.Inferno, "Pastel": pc.qualitative.Pastel,
    "Bold": pc.qualitative.Bold, "Set1": pc.qualitative.Set1,
    "Dark2": pc.qualitative.Dark2
}

formatos = {
    "PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"
}

# --- WIDGETS DE VISUALIZACIÓN ---
margen_dropdown = widgets.Dropdown(
    options=["Margen Izquierda (MI)", "Margen Derecha (MD)"],
    value="Margen Izquierda (MI)",
    description="Margen:"
)
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_dropdown = widgets.Dropdown(description="Variable:")
estilo_dropdown = widgets.Dropdown(
    options=estilos_grafico,
    value="Curvas suaves (spline)",
    description="Estilo gráfica:"
)
anio_dropdown = widgets.Dropdown(description="Año:")
tamanio_dropdown = widgets.Dropdown(
    options=list(tamanios_imagen.keys()),
    value="Mediano (900x500)",
    description="Tamaño:"
)
grosor_dropdown = widgets.Dropdown(
    options=list(grosores.keys()),
    value="Normal (2px)",
    description="Grosor línea:"
)
paleta_dropdown = widgets.Dropdown(
    options=list(paletas.keys()),
    value="Plotly",
    description="Paleta colores:"
)
formato_dropdown = widgets.Dropdown(
    options=list(formatos.keys()),
    value="PNG",
    description="Formato:"
)
ruta_text = widgets.Text(
    value="grafica_exportada",
    description="Ruta y nombre:",
    placeholder="ej: ./carpeta/mi_grafica"
)
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
output = widgets.Output()
output_guardar = widgets.Output()

# --- Función para actualizar opciones de dropdowns ---
def actualizar_opciones(change=None):
    margen = margen_dropdown.value
    df = df_mi if margen == "Margen Izquierda (MI)" else df_md
    if df.empty:
        punto_dropdown.options = ["Ninguno"]
        variable_dropdown.options = ["Ninguna"]
        anio_dropdown.options = ["Ninguno"]
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    columnas_excluidas = ['FECHA', 'INSTRUMENTO', 'MARGEN']
    variables = [col for col in df.select_dtypes(include='number').columns if col not in columnas_excluidas]
    puntos_fijos = sorted(df['INSTRUMENTO'].dropna().unique())
    anios = sorted(df['FECHA'].dt.year.dropna().unique().astype(int))
    punto_dropdown.options = ["Todos"] + list(puntos_fijos)
    punto_dropdown.value = "Todos"
    variable_dropdown.options = variables if variables else ["Ninguna"]
    variable_dropdown.value = variables[0] if variables else "Ninguna"
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

# --- Función para graficar ---
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        margen = margen_dropdown.value
        df = df_mi if margen == "Margen Izquierda (MI)" else df_md
        if df.empty:
            print(f"❌ No hay datos cargados para {margen}.")
            return
        variable = variable_dropdown.value
        if variable == "Ninguna":
            print("❌ No hay variables numéricas para graficar.")
            return
        estilo = estilo_dropdown.value
        punto = punto_dropdown.value
        anio = anio_dropdown.value
        ancho, alto = tamanios_imagen[tamanio_dropdown.value]
        grosor = grosores[grosor_dropdown.value]
        paleta = paletas[paleta_dropdown.value]
        titulo_dataset = f"Puntos Fijos {margen}"

        df_plot = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable])
        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
        if punto != "Todos":
            df_plot = df_plot[df_plot['INSTRUMENTO'] == punto]
        if df_plot.empty:
            print("No hay datos para graficar con la selección actual.")
            return

        fig = go.Figure()
        instrumentos = sorted(df_plot['INSTRUMENTO'].unique())
        color_map = {pf: paleta[i % len(paleta)] for i, pf in enumerate(instrumentos)}
        for pf in instrumentos:
            data_pf = df_plot[df_plot['INSTRUMENTO'] == pf]
            line_args = dict(width=grosor, color=color_map[pf])
            marker_args = dict(color=color_map[pf])
            if estilo == "Curvas suaves (spline)":
                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                    name=pf, line_shape="spline", line=line_args
                ))
            elif estilo == "Líneas rectas":
                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                    name=pf, line_shape="linear", line=line_args
                ))
            elif estilo == "Puntos":
                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'], y=data_pf[variable], mode="markers",
                    name=pf, marker=marker_args
                ))
            elif estilo == "Líneas + Puntos":
                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'], y=data_pf[variable], mode="lines+markers",
                    name=pf, line_shape="linear", line=line_args, marker=marker_args
                ))
            elif estilo == "Área apilada":
                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                    name=pf, stackgroup='one', line_shape="linear", line=line_args, marker=marker_args
                ))
            elif estilo == "Área + Líneas":
                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'], y=data_pf[variable], mode="lines",
                    name=pf, fill="tozeroy", line_shape="linear", line=line_args, marker=marker_args
                ))
            elif estilo == "Área + Líneas + Puntos":
                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'], y=data_pf[variable], mode="lines+markers",
                    name=pf, fill="tozeroy", line_shape="linear", line=line_args, marker=marker_args
                ))

        fig.update_layout(
            width=ancho, height=alto,
            title=f"{titulo_dataset}: {variable} en función del tiempo por Punto Fijo (PF)",
            xaxis_title="Fecha", yaxis_title=variable,
            legend_title="INSTRUMENTO", hovermode="x unified"
        )
        fig.show()

# --- Función para guardar gráfica ---
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formatos[formato_dropdown.value]
        ruta_archivo = ruta_text.value
        if not ruta_archivo.lower().endswith(ext):
            ruta_archivo += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Primero debes generar una gráfica.")
            return
        try:
            if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Para guardar como imagen/vector/pdf, instala 'kaleido':\n%pip install -U kaleido")
                    return
                fig.write_image(ruta_archivo, format=formato_dropdown.value.lower())
            elif formato_dropdown.value == "HTML":
                fig.write_html(ruta_archivo)
            else:
                print("❌ Tipo de archivo no soportado.")
                return
        except Exception as e:
            print("❌ Error al guardar la gráfica:", e)
            return
        print(f"✅ Gráfica guardada en: {os.path.abspath(ruta_archivo)}")
        if os.path.exists(ruta_archivo):
            if 'google.colab' in str(get_ipython()):
                from google.colab import files
                files.download(ruta_archivo)
            else:
                ruta_abs = os.path.abspath(ruta_archivo)
                display(HTML(f'<a href="file://{ruta_abs}" target="_blank">Descargar archivo</a>'))

# --- Configuración de eventos ---
margen_dropdown.observe(actualizar_opciones, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# --- Interfaz de usuario ---
titulo_html = "<h2 style='color:#1866a3; margin-bottom: 5px'>Visualización de Puntos Fijos</h2>"
display(HTML(titulo_html))
selectores = widgets.HBox([
    margen_dropdown, punto_dropdown, variable_dropdown, estilo_dropdown,
    anio_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown
])
display(selectores)
display(boton)
display(output)
controles_guardar = widgets.HBox([formato_dropdown, ruta_text])
display(controles_guardar)
display(boton_guardar)
display(output_guardar)

# --- Inicializar opciones ---
actualizar_opciones()

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()

In [ ]:
# --- LIMPIEZA Y PREPROCESAMIENTO ---
def limpiar_y_preparar(df):
    df = df.copy()

    # Normaliza columnas
    df.columns = df.columns.str.strip()

    # Convierte fechas
    if 'FECHA' in df.columns:
        df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
        df = df[df['FECHA'].notna()]

    return df

df_mi = limpiar_y_preparar(df_mi)
df_md = limpiar_y_preparar(df_md)

# --- RESÚMENES BÁSICOS ---
def resumen_df(df, nombre):
    print(f"📌 Resumen de {nombre}:")
    if df.empty:
        print("❌ No hay datos cargados.")
        return
    print(f"✔️ Total de filas: {len(df)}")
    print(f"🧭 Instrumentos únicos: {df['INSTRUMENTO'].nunique() if 'INSTRUMENTO' in df.columns else 'N/A'}")
    print(f"📆 Rango de fechas: {df['FECHA'].min().date()} → {df['FECHA'].max().date()}")
    print(f"📊 Columnas numéricas: {list(df.select_dtypes(include='number').columns)}")
    print()

resumen_df(df_mi, "Margen Izquierda (df_mi)")
resumen_df(df_md, "Margen Derecha (df_md)")


In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import io
import base64

# --- VARIABLES DISPONIBLES ---
variables_mi = [col for col in df_mi.select_dtypes(include='number').columns if col != 'FECHA']
variables_md = [col for col in df_md.select_dtypes(include='number').columns if col != 'FECHA']
puntos_mi = sorted(df_mi['INSTRUMENTO'].dropna().unique())
puntos_md = sorted(df_md['INSTRUMENTO'].dropna().unique())

# --- WIDGETS ---
margen_selector = widgets.Dropdown(options=['MI', 'MD'], description='Margen:')
punto_selector = widgets.Dropdown(description='Punto fijo:')
variable_selector = widgets.Dropdown(description='Variable:')
formato_selector = widgets.Dropdown(options=['Excel (.xlsx)', 'CSV (.csv)'], description='Formato:')
boton_analizar = widgets.Button(description="Ver análisis", button_style="primary")
boton_exportar = widgets.Button(description="Exportar", button_style="success")
output = widgets.Output()
link_descarga = widgets.HTML(value="")

# --- VARIABLE GLOBAL PARA GUARDAR RESULTADOS ---
datos_exportados = pd.DataFrame()

# --- ACTUALIZA OPCIONES SEGÚN MARGEN ---
def actualizar_opciones(*args):
    if margen_selector.value == 'MI':
        punto_selector.options = puntos_mi
        variable_selector.options = variables_mi
    else:
        punto_selector.options = puntos_md
        variable_selector.options = variables_md
actualizar_opciones()
margen_selector.observe(actualizar_opciones, names='value')

# --- FUNCIÓN DE ANÁLISIS ---
def analizar_datos(b):
    global datos_exportados
    with output:
        clear_output(wait=True)
        link_descarga.value = ""  # reset
        margen = margen_selector.value
        punto = punto_selector.value
        variable = variable_selector.value

        df = df_mi if margen == 'MI' else df_md
        datos = df[df['INSTRUMENTO'] == punto].sort_values('FECHA').copy()

        # Cálculos
        datos['valor_anterior'] = datos[variable].shift(1)
        datos['delta'] = datos[variable] - datos['valor_anterior']
        datos['% cambio'] = (datos['delta'] / datos['valor_anterior']) * 100
        datos['% cambio'] = datos['% cambio'].round(2)
        datos['alarma'] = np.where(datos['% cambio'].abs() > 10, '⚠️', '')

        columnas = ['FECHA', variable, 'valor_anterior', 'delta', '% cambio', 'alarma']
        datos_exportados = datos[columnas].reset_index(drop=True)
        display(datos_exportados)

        # Gráfico principal
        plt.figure(figsize=(12,4))
        plt.plot(datos['FECHA'], datos[variable], marker='o', label='Valor actual', color='blue')
        plt.plot(datos['FECHA'], datos['valor_anterior'], linestyle='--', label='Valor anterior', color='gray')
        plt.title(f'{variable} - Punto {punto} ({margen})')
        plt.xlabel("Fecha")
        plt.ylabel(variable)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Gráfico de delta
        plt.figure(figsize=(12,3))
        plt.plot(datos['FECHA'], datos['delta'], marker='o', color='purple', label='Delta')
        plt.axhline(0, color='gray', linestyle=':')
        if datos['alarma'].eq('⚠️').any():
            plt.scatter(datos.loc[datos['alarma'] == '⚠️', 'FECHA'],
                        datos.loc[datos['alarma'] == '⚠️', 'delta'],
                        color='red', label='Alarma >10%', s=60)
        plt.title(f'Variación entre lecturas ({variable})')
        plt.xlabel("Fecha")
        plt.ylabel("Delta")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

# --- FUNCIÓN DE EXPORTACIÓN Y DESCARGA ---
def exportar_datos(b):
    with output:
        clear_output(wait=True)
        if datos_exportados.empty:
            print("Primero debes ejecutar un análisis.")
            return

        formato = formato_selector.value
        punto = punto_selector.value
        variable = variable_selector.value

        if formato == 'CSV (.csv)':
            buffer = io.StringIO()
            datos_exportados.to_csv(buffer, index=False)
            content = buffer.getvalue()
            ext = "csv"
        else:
            buffer = io.BytesIO()
            with pd.ExcelWriter(buffer, engine='xlsxwriter') as writer:
                datos_exportados.to_excel(writer, index=False, sheet_name='Análisis')
            content = buffer.getvalue()
            ext = "xlsx"

        b64 = base64.b64encode(content if isinstance(content, bytes) else content.encode()).decode()
        filename = f'analisis_{punto}_{variable}.{ext}'
        href = f'<a download="{filename}" href="data:application/octet-stream;base64,{b64}" target="_blank">📥 Descargar {filename}</a>'
        link_descarga.value = href
        print("Archivo preparado para descarga.")

# --- EVENTOS ---
boton_analizar.on_click(analizar_datos)
boton_exportar.on_click(exportar_datos)

# --- MOSTRAR ---
display(widgets.HBox([margen_selector, punto_selector, variable_selector]))
display(widgets.HBox([formato_selector, boton_analizar, boton_exportar]))
display(output)
display(link_descarga)